# Character Classification
This notebook contains all steps of OCR

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

# Import Widgets
from ipywidgets import Button, Text, HBox, VBox
from IPython.display import display, clear_output

# Import costume functions, corresponding to notebooks
#from ocr import page, words, charSeg
from ocr.normalization import letterNorm
from ocr import charSeg
# Helpers
from ocr.helpers import implt, resize
from ocr.datahelpers import loadWordsData, char2idx, idx2char
from ocr.tfhelpers import Graph

### Global Variables

In [8]:
# Settings
LANG = 'cz'
LOC = 'models/char-clas/' + LANG + '/CharClassifier'

## Load Trained Model

In [9]:
charClass = Graph(LOC)

INFO:tensorflow:Restoring parameters from models/char-clas/cz/CharClassifier


## Load image

In [10]:
images, labels = loadWordsData('data/test_words', loadGaplines=False)

Loading words...
-> Number of words: 235


# Simple UI using widgets

In [11]:
class Cycler:
    """ Cycle through the words and recognise them """ 
    def __init__(self, images, labels, charClass, gapRNN=False, charRNN=False):
        self.images = images
        self.labels = labels
        self.charClass = charClass
        self.gapRNN = gapRNN
        self.charRNN = charRNN
        print(labels[0], ':', self.recogniseWord(images[0]))
        
    def recogniseWord(self, img):
        img = cv2.copyMakeBorder(img, 0, 0, 15, 15,
                                 cv2.BORDER_CONSTANT,
                                 value=[0, 0, 0])
        gaps = charSeg.segmentation(img, RNN=self.gapRNN)
        print(gaps)
        
        chars = []
        for i in range(len(gaps)-1):
            char = img[:, gaps[i]:gaps[i+1]]
            char, dim = letterNorm(char, dim=True)
            # TODO Test different values
            if dim[0] > 7 and dim[1] > 7:
                chars.append(char.flatten())
                
        chars = np.array(chars)
        
        if self.charRNN:
            pred = self.charClass.eval_feed({'inputs:0': chars,
                                             'length:0': [len(chars)]})
        else:
            pred = self.charClass.run(chars)
        
        word = ''
        for c in pred:
            # word += CHARS[charIdx]
            word += idx2char(c)
        return word
        

    def evaluate(self):
        """ Evaluate accuracy of the word classification """
        pass

In [12]:
# Class cycling through words
cycler = Cycler(images,
                labels,
                charClass,
                gapRNN=False,
                charRNN=False)

[17, 46, 87, 125]
Zda : pda
